In [ ]:
import astropy.coordinates as coord
from astropy.io import fits
from astropy.table import Table, vstack
import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.convolution import Gaussian2DKernel, convolve

from pyia import GaiaData
import gala.coordinates as gc
from scipy.ndimage import gaussian_filter
from scipy.stats import binned_statistic_2d
from gala.mpl_style import hesperia, laguna, center_emph, center_deemph
from tqdm import tqdm
import dustmaps
from dustmaps.sfd import SFDQuery

In [ ]:
# import dustmaps.sfd
# dustmaps.sfd.fetch()

In [ ]:
L_lim = [-180, 180] * u.deg
B_lim = [-80, 80] * u.deg

In [ ]:
import scipy.stats

def get_ext(G, bp, rp, ebv, maxnit=10):
    """ Compute the Gaia extinctions assuming relations from Babusieux
    Arguments: G, bp, rp, E(B-V)
    maxnit -- number of iterations
    Returns extinction in G,bp, rp
    Author: Sergey Koposov skoposov@cmu.edu
    """
    c1, c2, c3, c4, c5, c6, c7 = [0.9761, -0.1704,
                                  0.0086, 0.0011, -0.0438, 0.0013, 0.0099]
    d1, d2, d3, d4, d5, d6, d7 = [
        1.1517, -0.0871, -0.0333, 0.0173, -0.0230, 0.0006, 0.0043]
    e1, e2, e3, e4, e5, e6, e7 = [
        0.6104, -0.0170, -0.0026, -0.0017, -0.0078, 0.00005, 0.0006]
    A0 = 3.1*ebv
    P1 = np.poly1d([c1, c2, c3, c4][::-1])

    def F1(bprp): return np.poly1d(
        [c1, c2, c3, c4][::-1])(bprp)+c5*A0+c6*A0**2+c7*bprp*A0

    def F2(bprp): return np.poly1d(
        [d1, d2, d3, d4][::-1])(bprp)+d5*A0+d6*A0**2+d7*bprp*A0

    def F3(bprp): return np.poly1d(
        [e1, e2, e3, e4][::-1])(bprp)+e5*A0+e6*A0**2+e7*bprp*A0
    xind = np.isfinite(bp+rp+G)
    curbp = bp-rp
    for i in range(maxnit):
        AG = F1(curbp)*A0
        Abp = F2(curbp)*A0
        Arp = F3(curbp)*A0
        curbp1 = bp-rp-Abp+Arp
        delta = np.abs(curbp1-curbp)[xind]
        print(scipy.stats.scoreatpercentile(delta, 99))
        curbp = curbp1
    AG = F1(curbp)*A0
    Abp = F2(curbp)*A0
    Arp = F3(curbp)*A0
    return AG, Abp, Arp

In [ ]:
# M_G = 2*u.mag
M_G = -1*u.mag
coord.Distance(30*u.kpc).distmod + M_G

### SFD map of region

In [ ]:
LL = np.arange(L_lim[0].value, L_lim[1].value+0.1, 0.1)
BB = np.arange(B_lim[0].value, B_lim[1].value+0.1, 0.1)
LL, BB = np.meshgrid(LL, BB)
coords = coord.SkyCoord(LL*u.deg, BB*u.deg,
                        distance=1.*u.kpc, frame=gc.MagellanicStream).transform_to(coord.Galactic)

In [ ]:
sfd = SFDQuery()
EBV_grid_sfd = sfd(coords)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

c = ax.pcolormesh(LL, BB, 
   EBV_grid_sfd,
   cmap='binary',
   norm=mpl.colors.LogNorm(vmin=1E-2, vmax=1e0)
)

ax.set_xlim(120, -140)
ax.set_ylim(-60, 60)

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$B_{\rm MS}$ [deg]')
fig.tight_layout()

fig.savefig('../plots/sfd_mag.png', dpi=250)

BP-RP and G mag cuts to get ~A/B stars @ 30 kpc

In [ ]:
# g = GaiaData('/Users/adrian/data/GaiaDR2/blue_distant.fits')
g = GaiaData('/Users/adrian/Downloads/blue_distant_moar.fits')
g = g[g.parallax < 0.5*u.mas]
len(g)

In [ ]:
c = coord.SkyCoord(ra=g.ra, dec=g.dec,
                   distance=np.full(len(g), 35)*u.kpc,
                   pm_ra_cosdec=g.pmra,
                   pm_dec=g.pmdec,
                   radial_velocity=np.zeros(len(g))*u.km/u.s)

In [ ]:
EBV = sfd(c)

In [ ]:
G = g.phot_g_mean_mag.value
bp = g.phot_bp_mean_mag.value
rp = g.phot_rp_mean_mag.value
A_G, A_BP, A_RP = get_ext(G, bp, rp, EBV, 25)

In [ ]:
ext_mask = (np.isfinite(A_G) & np.isfinite(A_BP) & np.isfinite(A_RP) & (A_G < 4) & 
            (g.phot_g_mean_mag < 20.2*u.mag))
ext_mask.sum()

In [ ]:
gal_c = c.galactic
sag_c = c.transform_to(gc.Sagittarius)

In [ ]:
mag_c = c.transform_to(gc.MagellanicStream)
mag_c = gc.reflex_correct(mag_c)

In [ ]:
L = mag_c.L.wrap_at(180*u.deg)
B = mag_c.B

lmc_mask = (np.sqrt(L**2 + (B-4*u.deg)**2) > 10*u.deg) # Remove LMC
bridge_mask = (np.sqrt((L+10*u.deg)**2 + (B+6*u.deg)**2) > 4*u.deg) # Remove bridge
smc_mask = (np.sqrt((L+15*u.deg)**2 + (B+11*u.deg)**2) > 5*u.deg) # Remove SMC    
mag_mask = lmc_mask & smc_mask & bridge_mask
    
lmc_ann_mask = ((np.sqrt(L**2 + (B-4*u.deg)**2) > 5*u.deg) & 
                (np.sqrt(L**2 + (B-4*u.deg)**2) < 30*u.deg))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))

ax.plot(mag_c.L.wrap_at(180*u.deg).degree[ext_mask & mag_mask], 
        mag_c.B.degree[ext_mask & mag_mask], 
        linestyle='none', marker=',', alpha=0.2)

ax.set_xlim(L_lim[1].to(u.degree).value, L_lim[0].to(u.degree).value)
ax.set_ylim(B_lim[0].to(u.degree).value, B_lim[1].to(u.degree).value)

# --- 

fig, ax = plt.subplots(1, 1, figsize=(15, 5))

ax.plot(c.ra.degree[ext_mask & mag_mask], 
        c.dec.degree[ext_mask & mag_mask], 
        linestyle='none', marker=',', alpha=0.2)

# ax.set_xlim(L_lim[1].to(u.degree).value, L_lim[0].to(u.degree).value)
# ax.set_ylim(B_lim[0].to(u.degree).value, B_lim[1].to(u.degree).value)

# --- 

fig, ax = plt.subplots(1, 1, figsize=(15, 5))

ax.plot(c.galactic.l.wrap_at(180*u.deg).degree[ext_mask & mag_mask], 
        c.galactic.b.degree[ext_mask & mag_mask], 
        linestyle='none', marker=',', alpha=0.2)

# ax.set_xlim(L_lim[1].to(u.degree).value, L_lim[0].to(u.degree).value)
# ax.set_ylim(B_lim[0].to(u.degree).value, B_lim[1].to(u.degree).value)

In [ ]:
bprp0 = (g.phot_bp_mean_mag - g.phot_rp_mean_mag) - (A_BP - A_RP)*u.mag
G0 = g.phot_g_mean_mag - A_G*u.mag

First, to get a sense, we'll plot a CMD of LMC stars selected from an outer annulus:

In [ ]:
spur1_mask = (bprp0 > -0.5*u.mag) & (G0 < (20*bprp0 + 17.5*u.mag)) & (G0 < 19*u.mag)
spur2_mask = (G0 < (20*(bprp0+1*u.mag) + 5.*u.mag)) & (bprp0 < 0.5*u.mag) & (G0 > 17*u.mag)
spur3_mask = (bprp0 > -1.1*u.mag) & (bprp0 < -.4*u.mag) & (G0 > (10*(bprp0+1*u.mag) + 12.*u.mag))
spur4_mask = (bprp0 < -1*u.mag) & (G0 > (10*(bprp0+1*u.mag) + 16*u.mag))

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

# ax.plot(bprp0[lmc_ann_mask], G0[lmc_ann_mask], 
#         marker='.', linestyle='none', alpha=0.2)
H, xe, ye = np.histogram2d(bprp0[lmc_ann_mask & smc_mask], G0[lmc_ann_mask & smc_mask],
                           bins=(np.arange(-2, 0.8+1E-3, 0.05),
                                 np.arange(10, 20.5, 0.05)))

ax.pcolormesh(xe, ye, np.log10(H.T))

# ax.plot(bprp0[lmc_ann_mask & spur1_mask], G0[lmc_ann_mask & spur1_mask], 
#         marker='.', linestyle='none', alpha=0.2)

# ax.plot(bprp0[lmc_ann_mask & spur2_mask], G0[lmc_ann_mask & spur2_mask], 
#         marker='.', linestyle='none', alpha=0.02)

# ax.plot(bprp0[lmc_ann_mask & spur3_mask], G0[lmc_ann_mask & spur3_mask], 
#         marker='.', linestyle='none', alpha=0.2)

# ax.plot(bprp0[lmc_ann_mask & spur4_mask], G0[lmc_ann_mask & spur4_mask], 
#         marker='.', linestyle='none', alpha=0.2)

# ax.set_aspect('equal')

ax.axhline(coord.Distance(50*u.kpc).distmod.value + 1.)

ax.set_xlim(-1, 0.8)
ax.set_ylim(20, 12)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 5), sharex=True, sharey=True)

axes[0].plot(mag_c.pm_L_cosB[lmc_ann_mask & spur1_mask], mag_c.pm_B[lmc_ann_mask & spur1_mask], 
             linestyle='none', marker=',', alpha=0.2)

axes[1].plot(mag_c.pm_L_cosB[lmc_ann_mask & spur2_mask], mag_c.pm_B[lmc_ann_mask & spur2_mask], 
             linestyle='none', marker=',', alpha=0.2)

axes[2].plot(mag_c.pm_L_cosB[lmc_ann_mask & spur3_mask], mag_c.pm_B[lmc_ann_mask & spur3_mask], 
             linestyle='none', marker=',', alpha=0.2)

axes[3].plot(mag_c.pm_L_cosB[lmc_ann_mask & spur4_mask], mag_c.pm_B[lmc_ann_mask & spur4_mask], 
             linestyle='none', marker=',', alpha=0.2)


fig, axes = plt.subplots(1, 4, figsize=(20, 5), sharex=True, sharey=True)

axes[0].plot(mag_c.L.wrap_at(180*u.deg).degree[lmc_ann_mask & spur1_mask], 
             mag_c.B.degree[lmc_ann_mask & spur1_mask], 
             linestyle='none', marker='.', alpha=0.1)

axes[1].plot(mag_c.L.wrap_at(180*u.deg).degree[lmc_ann_mask & spur2_mask], 
             mag_c.B.degree[lmc_ann_mask & spur2_mask], 
             linestyle='none', marker='.', alpha=0.1)

axes[2].plot(mag_c.L.wrap_at(180*u.deg).degree[lmc_ann_mask & spur3_mask], 
             mag_c.B.degree[lmc_ann_mask & spur3_mask], 
             linestyle='none', marker='.', alpha=0.8)

axes[3].plot(mag_c.L.wrap_at(180*u.deg).degree[lmc_ann_mask & spur4_mask], 
             mag_c.B.degree[lmc_ann_mask & spur4_mask], 
             linestyle='none', marker='.', alpha=0.8)

---

In [ ]:
g2 = GaiaData.from_query('''SELECT *
FROM gaiadr2.gaia_source
WHERE parallax < 1 AND bp_rp > -1 AND bp_rp < 2 AND
      CONTAINS(POINT('ICRS', ra, dec), 
               CIRCLE('ICRS', 78.52816666666665, -40.04655555555556, 0.08333333333333333))=1 AND
      (pmra > -15) AND (pmra < 15) AND 
      (pmdec > -15) AND (pmdec < 15)
''')

In [ ]:
ngc_c = coord.SkyCoord(ra=g2.ra, dec=g2.dec, distance=100*u.kpc)
ngc_EBV = sfd(ngc_c)
G = g2.phot_g_mean_mag.value
bp = g2.phot_bp_mean_mag.value
rp = g2.phot_rp_mean_mag.value
ngc_A_G, ngc_A_BP, ngc_A_RP = get_ext(G, bp, rp, ngc_EBV)

ngc_bprp0 = (g2.phot_bp_mean_mag - g2.phot_rp_mean_mag) - (ngc_A_BP - ngc_A_RP)*u.mag
ngc_G0 = g2.phot_g_mean_mag - ngc_A_G*u.mag

In [ ]:
ngc_MG0 = ngc_G0 - coord.Distance(12.1*u.kpc).distmod
# H, xe, ye = np.histogram2d(ngc_bprp0, ngc_G0,
#                            bins=(np.arange(-1, 1.5+1E-3, 0.05),
#                                  np.arange(8, 20.5, 0.05)))

fig, ax = plt.subplots(1, 1, figsize=(6, 6))

# ax.pcolormesh(xe, ye, np.log10(H.T))
ax.scatter(ngc_bprp0, ngc_MG0)
ax.set_ylim(5, -3)

---

In [ ]:
# more_mask = (G0 > 16.*u.mag) & (bprp0 > -0.5*u.mag)
# more_mask = (bprp0 < 0*u.mag) & (bprp0 > -0.4*u.mag)
# more_mask = (bprp0 < 0.35*u.mag) & (bprp0 > -0*u.mag) & (G0 > (2.5/0.5*bprp0 + 17.5*u.mag))


fig, ax = plt.subplots(1, 1, figsize=(6, 6))

H, xe, ye = np.histogram2d(bprp0[ext_mask & mag_mask], G0[ext_mask & mag_mask],
                           bins=(np.arange(-2, 0.8+1E-3, 0.05),
                                 np.arange(10, 20.5, 0.05)))

ax.pcolormesh(xe, ye, np.log10(H.T))

for d in [25, 50, 75]*u.kpc:
    ax.axhline(coord.Distance(d).distmod.value + 1.)
    
    
# ATTEMPT 1: BHB?!
# xx = np.linspace(-1, 0.8, 128)
# ax.plot(xx, 30*(xx+0.1)+16)
# ax.plot(xx, 30*(xx-0.2)+16)

# more_mask = ((G0 < 19.5*u.mag) & (G0 > 18.*u.mag) &
#              (G0 < (30*(bprp0+0.1*u.mag)+16*u.mag)) & 
#              (G0 > (30*(bprp0-0.2*u.mag)+16*u.mag)))

# ATTEMPT 2: redder HB??
# # - ah: these are like red clump stars? so really make outer disk features pop...
# more_mask = ((G0 < 20*u.mag) & (G0 > 15.*u.mag) &
#              (bprp0 > 0.5*u.mag) & (bprp0 < 0.8*u.mag))


# ATTEMPT 3: now trying HB generically
# more_mask = ((G0 < 20*u.mag) & (G0 > 17.5*u.mag) &
#              (bprp0 > -0.15*u.mag) & (bprp0 < 0.1*u.mag))
more_mask = ((bprp0 < -0.5*u.mag) & (bprp0 > -1.5*u.mag) & 
             (G0 > 17.*u.mag))

# YE OLDE SELECTION:
# more_mask = ((bprp0 < 0.1*u.mag) & (bprp0 > -0.7*u.mag) & 
#              (G0 > (2.5/0.5*bprp0 + 17.5*u.mag)) &
#              (G0 > 17.5*u.mag))


ax.plot(bprp0[ext_mask & mag_mask & more_mask], 
        G0[ext_mask & mag_mask & more_mask], 
        marker=',', linestyle='none', alpha=1, color='tab:orange')

ax.scatter(ngc_bprp0, ngc_MG0 + coord.Distance(50*u.kpc).distmod, color='k', zorder=100)
ax.scatter(ngc_bprp0, ngc_MG0 + coord.Distance(25*u.kpc).distmod, color='k', zorder=100)

ax.set_xlim(-1.5, 0.8)
ax.set_ylim(20, 12)
# ax.set_ylim(12, 20)

In [ ]:
all_mask = mag_mask & ext_mask & more_mask

In [ ]:
# pm_mask = (mag_c.pm_B > -0.1*u.mas/u.yr) & (mag_c.pm_B < 3*u.mas/u.yr)
# pm_mask = np.abs(mag_c.pm_B) < 0.6*u.mas/u.yr
# pm_mask = np.ones(len(mag_c)).astype(bool)
# pm_mask = np.sqrt((mag_c.pm_L_cosB - -2.3*u.mas/u.yr)**2 + (mag_c.pm_B - -1.5*u.mas/u.yr)**2) < 0.5*u.mas/u.yr

left_mask = (((mag_c.B > 30*u.deg) & (mag_c.B < 60*u.deg) &
              (mag_c.L.wrap_at(180*u.deg) > -110*u.deg) & (mag_c.L.wrap_at(180*u.deg) < -20*u.deg)) |
             ((mag_c.B > -60*u.deg) & (mag_c.B < -20*u.deg) &
              (mag_c.L.wrap_at(180*u.deg) > 50*u.deg) & (mag_c.L.wrap_at(180*u.deg) < 100*u.deg))
             & ext_mask & more_mask)

In [ ]:
pm_bins = np.arange(-4, 4+0.1, 0.1)

maggg = (ext_mask & more_mask & (~lmc_mask | ~smc_mask))
mag_pm, pm_x, pm_y = np.histogram2d(c.pm_ra_cosdec[maggg].value, 
                                    c.pm_dec[maggg].value,
                                    bins=(pm_bins, pm_bins))

fg_pm, pm_x, pm_y = np.histogram2d(c.pm_ra_cosdec[all_mask].value, 
                                   c.pm_dec[all_mask].value,
                                   bins=(pm_bins, pm_bins))

bg_pm, pm_x, pm_y = np.histogram2d(c.pm_ra_cosdec[left_mask].value, 
                                   c.pm_dec[left_mask].value,
                                   bins=(pm_bins, pm_bins))

fig, axes = plt.subplots(1, 4, figsize=(16, 5), sharex=True, sharey=True)

axes[0].pcolormesh(pm_x, pm_y, mag_pm.T)
axes[0].set_title('LMC/SMC')

axes[1].pcolormesh(pm_x, pm_y, fg_pm.T)
axes[1].set_title('fg')

axes[2].pcolormesh(pm_x, pm_y, bg_pm.T)
axes[2].set_title('bg')

axes[3].pcolormesh(pm_x, pm_y, fg_pm.T/(57600) - bg_pm.T/4700.)

fig.tight_layout()

In [ ]:
pm_bins = np.arange(-4, 4+0.1, 0.1)

maggg = (ext_mask & more_mask & (~lmc_mask | ~smc_mask))
mag_pm, pm_x, pm_y = np.histogram2d(gal_c.pm_l_cosb[maggg].value, 
                                    gal_c.pm_b[maggg].value,
                                    bins=(pm_bins, pm_bins))

fg_pm, pm_x, pm_y = np.histogram2d(gal_c.pm_l_cosb[all_mask].value, 
                                   gal_c.pm_b[all_mask].value,
                                   bins=(pm_bins, pm_bins))

bg_pm, pm_x, pm_y = np.histogram2d(gal_c.pm_l_cosb[left_mask].value, 
                                   gal_c.pm_b[left_mask].value,
                                   bins=(pm_bins, pm_bins))

fig, axes = plt.subplots(1, 4, figsize=(16, 5), sharex=True, sharey=True)

axes[0].pcolormesh(pm_x, pm_y, mag_pm.T)
axes[0].set_title('LMC/SMC')

axes[1].pcolormesh(pm_x, pm_y, fg_pm.T)
axes[1].set_title('fg')

axes[2].pcolormesh(pm_x, pm_y, bg_pm.T)
axes[2].set_title('bg')

axes[3].pcolormesh(pm_x, pm_y, fg_pm.T/(57600) - bg_pm.T/4700.)

fig.tight_layout()

In [ ]:
pm_bins = np.arange(-5, 5+0.1, 0.1)

maggg = (ext_mask & more_mask & (~lmc_mask | ~smc_mask))
mag_pm, pm_x, pm_y = np.histogram2d(mag_c.pm_L_cosB[maggg].value, 
                                    mag_c.pm_B[maggg].value,
                                    bins=(pm_bins, pm_bins))

fg_pm, pm_x, pm_y = np.histogram2d(mag_c.pm_L_cosB[all_mask].value, 
                                   mag_c.pm_B[all_mask].value,
                                   bins=(pm_bins, pm_bins))

bg_pm, pm_x, pm_y = np.histogram2d(mag_c.pm_L_cosB[left_mask].value, 
                                   mag_c.pm_B[left_mask].value,
                                   bins=(pm_bins, pm_bins))

fig, axes = plt.subplots(1, 4, figsize=(16, 5), sharex=True, sharey=True)

axes[0].pcolormesh(pm_x, pm_y, mag_pm.T)#, norm=mpl.colors.LogNorm())
axes[0].set_title('LMC/SMC')

axes[1].pcolormesh(pm_x, pm_y, fg_pm.T)#, norm=mpl.colors.LogNorm())
axes[1].set_title('fg')

axes[2].pcolormesh(pm_x, pm_y, bg_pm.T)#, norm=mpl.colors.LogNorm())
axes[2].set_title('bg')

axes[3].pcolormesh(pm_x, pm_y, fg_pm.T/(57600) - bg_pm.T/4700.)

for ax in axes: 
    r = mpl.patches.Ellipse((0.9, 0.1), width=2, height=2.2, facecolor='none', edgecolor='tab:red')
    ax.add_patch(r)
    
for ax in axes: 
    r = mpl.patches.Ellipse((-1.2, 0.1), width=2, height=2.2, facecolor='none', edgecolor='tab:red')
    ax.add_patch(r)

fig.tight_layout()

In [ ]:
def in_ellipse(pts, ctr, width, height):
    rx = width/2
    ry = height/2
    return np.sqrt(((pts[0] - ctr[0])/rx)**2 + ((pts[1] - ctr[1])/ry)**2) < 1

In [ ]:
# Sgr:
# pm_mask = np.sqrt((mag_c.pm_L_cosB - 0.2*u.mas/u.yr)**2 + 
#                   (mag_c.pm_B - 2.5*u.mas/u.yr)**2) < 1.2*u.mas/u.yr
# pm_mask |= np.sqrt((mag_c.pm_L_cosB - 1*u.mas/u.yr)**2 + 
#                    (mag_c.pm_B - -2.*u.mas/u.yr)**2) < 1.2*u.mas/u.yr

# Mag stream?!
pts = np.vstack((mag_c.pm_L_cosB.value, mag_c.pm_B.value))
pm_mask = (in_ellipse(pts, [0.9, 0.1], 2, 2.2) & 
           (c.pm_ra_cosdec > -0.2*u.mas/u.yr))

pm_control_mask = (in_ellipse(pts, [-1.2, 0.1], 2, 2.2) & 
                   (c.pm_ra_cosdec > -0.2*u.mas/u.yr))

# pm_mask = np.ones(len(g)).astype(bool)
pm_mask.sum(), pm_control_mask.sum()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

H, xe, ye = np.histogram2d(bprp0[mag_mask & ext_mask & pm_mask], 
                           G0[mag_mask & ext_mask & pm_mask],
                           bins=(np.arange(-2, 0.8+1E-3, 0.05),
                                 np.arange(10, 20.5, 0.05)))

ax.pcolormesh(xe, ye, np.log10(H.T))

for d in [20, 50, 75]*u.kpc:
    ax.axhline(coord.Distance(d).distmod.value + 1.)

xxx = np.linspace(-1.5, 1.5, 128)
ax.plot(xxx, 25*(xxx-0.3) + 18.7)
ax.plot(xxx, 10*(xxx+0.4) + 18.7)
ax.plot(xxx, -2*(xxx+0.) + 19.4)
ax.plot(xxx, -20*(xxx+0.25) + 16)

# more_more_mask = ((G0 > (10*(bprp0-0.*u.mag) + 18.7*u.mag)) & 
#                   (G0 < (10*(bprp0+0.5*u.mag) + 18.7*u.mag)) & 
#                   (G0 < 20*u.mag) & (G0 > 17.5*u.mag))

more_more_mask = ((G0 > (25*(bprp0-0.35*u.mag) + 18.7*u.mag)) & 
                  (G0 < (10*(bprp0+0.4*u.mag) + 18.7*u.mag)) & 
                  (bprp0 < 0.3*u.mag) &
                  (G0 < (-2*(bprp0+0.*u.mag) + 19.4*u.mag)) & 
                  (G0 > (-20*(bprp0+0.25*u.mag) + 16*u.mag)) &
                  (G0 < 19.5*u.mag) & (G0 > 16.5*u.mag))
    
ax.plot(bprp0[ext_mask & mag_mask & more_more_mask], 
        G0[ext_mask & mag_mask & more_more_mask], 
        marker=',', linestyle='none', alpha=1, color='tab:orange')

# ax.scatter(ngc_bprp0, ngc_MG0 + coord.Distance(50*u.kpc).distmod, color='k', zorder=100)
# ax.scatter(ngc_bprp0, ngc_MG0 + coord.Distance(20*u.kpc).distmod, color='k', zorder=100)

ax.set_xlim(-1.5, 0.8)
# ax.set_ylim(20, 12)
ax.set_ylim(12, 20)

In [ ]:
all_mask = mag_mask & ext_mask & more_more_mask

In [ ]:
ol_blob = coord.SkyCoord(ra=178.94197852110662*u.deg,
                         dec=-29.46957451372061*u.deg)
ol_blob = ol_blob.transform_to(gc.MagellanicStream)

In [ ]:
bins = (np.arange(L_lim[0].value, L_lim[1].value+1, 3),
        np.arange(B_lim[0].value, B_lim[1].value+1, 3))

for cc in [coord.SkyCoord(mag_c), c, gal_c, sag_c]:
    lon_name, lat_name, _ = list(cc.representation_component_names.keys())
    
    fig, ax = plt.subplots(1, 1, figsize=(15, 7))
    
    bg, xe, ye = np.histogram2d(getattr(cc, lon_name).wrap_at(180*u.deg).degree[all_mask & pm_control_mask], 
                                getattr(cc, lat_name).degree[all_mask & pm_control_mask],
                                bins=bins)
    # gauss = Gaussian2DKernel(0.5)
    # bg = convolve(bg, gauss)
    
    H, xe, ye = np.histogram2d(getattr(cc, lon_name).wrap_at(180*u.deg).degree[all_mask & pm_mask], 
                               getattr(cc, lat_name).degree[all_mask & pm_mask],
                               bins=bins)
    
    # gauss = Gaussian2DKernel(0.5)
    # H = convolve(H, gauss)
    ax.pcolormesh(xe, ye, (H - bg).T, 
                  norm=mpl.colors.LogNorm(vmin=0.3, vmax=10),
                  cmap='Greys')

    # gas = [[(35, 5), (30, 5)],
    #        [(35, -17), (30, 7)],
    #        [(10, -32), (20, 17)],
    #        [(-35, -25), (15, 35)],
    #        [(-110, -10), (80, 20)]]
    # # for row in gas:
    # #     r = mpl.patches.Rectangle(row[0], row[1][0], row[1][1],
    # #                               facecolor='none', edgecolor='tab:red')
    # #     ax.add_patch(r)

    # r = mpl.patches.Circle((ol_blob.L.degree, ol_blob.B.degree), radius=2., 
    #                        facecolor='none', edgecolor='tab:green')
    # ax.add_patch(r)

    ax.set_xlim(L_lim[1].to(u.degree).value, L_lim[0].to(u.degree).value)
    ax.set_ylim(B_lim[0].to(u.degree).value, B_lim[1].to(u.degree).value)

    ax.set_xlabel(r'${0}$ [deg]'.format(lon_name))
    ax.set_ylabel(r'${0}$ [deg]'.format(lat_name))

    fig.tight_layout()

#     fig.savefig('../plots/mag_{0}.png'.format(cc.frame.name), dpi=250)

In [ ]:
for cc in [mag_c, c, c.galactic]:
    lon_name, lat_name, _ = list(cc.representation_component_names.keys())
    
    fig, ax = plt.subplots(1, 1, figsize=(15, 7))
    
    stat = binned_statistic_2d(getattr(cc, lon_name).wrap_at(180*u.deg).degree[all_mask & pm_mask],
                               getattr(cc, lat_name).degree[all_mask & pm_mask],
                               G0[all_mask & pm_mask],
                               bins=(np.arange(L_lim[0].value, L_lim[1].value+1, 2),
                                     np.arange(B_lim[0].value, B_lim[1].value+1, 2)),
                               statistic='count')
    H = stat.statistic.T
    # H = gaussian_filter(H, sigma=0.7)
    col = ax.pcolormesh(stat.x_edge, stat.y_edge, H,
                        vmin=0, vmax=10, cmap='viridis')
    fig.colorbar(col)

    ax.set_xlim(L_lim[1].to(u.degree).value, L_lim[0].to(u.degree).value)
    ax.set_ylim(B_lim[0].to(u.degree).value, B_lim[1].to(u.degree).value)

    ax.set_xlabel(r'${0}$ [deg]'.format(lon_name))
    ax.set_ylabel(r'${0}$ [deg]'.format(lat_name))

    fig.tight_layout()

    # fig.savefig('../plots/mag_blue.png', dpi=250)
    
    break

In [ ]:
for cc in [mag_c, c, c.galactic]:
    lon_name, lat_name, _ = list(cc.representation_component_names.keys())
    
    fig, ax = plt.subplots(1, 1, figsize=(15, 7))
    
    stat = binned_statistic_2d(getattr(cc, lon_name).wrap_at(180*u.deg).degree[all_mask & pm_mask],
                               getattr(cc, lat_name).degree[all_mask & pm_mask],
                               G0[all_mask & pm_mask],
                               bins=(np.arange(L_lim[0].value, L_lim[1].value+1, 1),
                                     np.arange(B_lim[0].value, B_lim[1].value+1, 1)),
                               statistic=np.nanmedian)
    H = stat.statistic.T
    col = ax.pcolormesh(stat.x_edge, stat.y_edge, H,
                        vmin=19.1, vmax=19.8, cmap='Spectral')

    ax.set_xlim(L_lim[1].to(u.degree).value, L_lim[0].to(u.degree).value)
    ax.set_ylim(B_lim[0].to(u.degree).value, B_lim[1].to(u.degree).value)

    ax.set_xlabel(r'${0}$ [deg]'.format(lon_name))
    ax.set_ylabel(r'${0}$ [deg]'.format(lat_name))

    fig.tight_layout()

    # fig.savefig('../plots/mag_blue.png', dpi=250)
    
    break

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

ax.plot(mag_c.L.wrap_at(180*u.deg).degree[all_mask & pm_mask], 
        mag_c.B.degree[all_mask & pm_mask],
        marker='.', linestyle='none', alpha=0.5)

gas = [[(35, 5), (30, 5)],
       [(35, -17), (30, 7)],
       [(10, -32), (20, 17)],
       [(-35, -25), (15, 35)],
       [(-110, -10), (80, 20)]]
for row in gas:
    r = mpl.patches.Rectangle(row[0], row[1][0], row[1][1],
                              facecolor='none', edgecolor='tab:red')
    ax.add_patch(r)

r = mpl.patches.Circle((ol_blob.L.degree, ol_blob.B.degree), radius=2., 
                       facecolor='none', edgecolor='tab:green')
ax.add_patch(r)
    
ax.set_xlim(120, -140)
ax.set_ylim(-60, 60)

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$B_{\rm MS}$ [deg]')

# fig.savefig('mag_blue.png', dpi=250)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

ax.plot(c.ra.degree[all_mask & pm_mask], 
        c.dec.degree[all_mask & pm_mask],
        marker='.', linestyle='none', alpha=0.5)
    
ax.set_xlim(360, 0)
# ax.set_ylim(-60, 60)

# ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
# ax.set_ylabel(r'$B_{\rm MS}$ [deg]')

In [ ]:
stat_muL = binned_statistic_2d(mag_c.L.wrap_at(180*u.deg).degree[all_mask & pm_mask],
                               mag_c.B.degree[all_mask & pm_mask],
                               mag_c.pm_L_cosB[all_mask & pm_mask],
                               bins=(np.arange(-140, 120+1, 2),
                                     np.arange(-60, 60+1, 2)),
                               statistic=np.nanmean)

stat_muB = binned_statistic_2d(mag_c.L.wrap_at(180*u.deg).degree[all_mask & pm_mask],
                               mag_c.B.degree[all_mask & pm_mask],
                               mag_c.pm_B[all_mask & pm_mask],
                               bins=(np.arange(-140, 120+1, 2),
                                     np.arange(-60, 60+1, 2)),
                               statistic=np.nanmean)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

# H = gaussian_filter(H, sigma=0.75)
cc = ax.pcolormesh(stat_muL.x_edge, stat_muL.y_edge, stat_muL.statistic.T,
                   vmin=-2, vmax=2, cmap=center_emph)
cb = fig.colorbar(cc)
cb.set_label(r'${\rm mean}(\mu_L \cos B)$')

ax.set_xlim(120, -140)
ax.set_ylim(-60, 60)

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$B_{\rm MS}$ [deg]')

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

# H = gaussian_filter(H, sigma=0.75)
cc = ax.pcolormesh(stat_muB.x_edge, stat_muB.y_edge, stat_muB.statistic.T,
                   vmin=-1, vmax=1, cmap=center_emph)
cb = fig.colorbar(cc)
cb.set_label(r'${\rm mean}(\mu_B)$')

ax.set_xlim(120, -140)
ax.set_ylim(-60, 60)

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$B_{\rm MS}$ [deg]')

fig.tight_layout()
# fig.savefig('mag_pm.png', dpi=250)

---
# WTF?! TriAnd and outer disk??

In [ ]:
mgiants = Table.read('../../low-latitude-structures/data/mgiants-only.csv', format='ascii.csv')

In [ ]:
cc = c.galactic
lon_name, lat_name, _ = list(cc.representation_component_names.keys())

stat = binned_statistic_2d(getattr(cc, lon_name).wrap_at(360*u.deg).degree[all_mask & pm_mask],
                           getattr(cc, lat_name).degree[all_mask & pm_mask],
                           G0[all_mask & pm_mask],
                           bins=(np.arange(0, 360+1, 1),
                                 np.arange(B_lim[0].value, B_lim[1].value+1, 1)),
                           statistic='mean')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7))

H = stat.statistic.T
# gauss = Gaussian2DKernel(0.75)
# H = convolve(H, gauss)
col = ax.pcolormesh(stat.x_edge, stat.y_edge, H, 
                    vmin=17.3, vmax=19.5,
                    cmap='hesperia_r')
fig.colorbar(col)

for i, (name, col) in enumerate(zip(np.unique(mgiants['structure']),
                                    ['tab:red', 'tab:green', 'tab:blue'])):
    structure = mgiants['structure'] == name
    ax.scatter(mgiants['l'][structure], mgiants['b'][structure], c=col,
               marker='.', label=name)
ax.legend(loc='upper left')

# ax.set_xlim(L_lim[1].to(u.degree).value, L_lim[0].to(u.degree).value)
ax.set_xlim(360, 0)
ax.set_ylim(B_lim[0].to(u.degree).value, B_lim[1].to(u.degree).value)

ax.set_xlabel(r'${0}$ [deg]'.format(lon_name))
ax.set_ylabel(r'${0}$ [deg]'.format(lat_name))

fig.tight_layout()

#     fig.savefig('../plots/mag_{0}.png'.format(cc.frame.name), dpi=250)